In [ ]:
from IPython.core.display import HTML
with open ("style.css", "r") as file:
    css = file.read()
HTML(css)

# Minimizing a <span style="font-variant:small-caps;">Fsm</span>

In [ ]:
%run FixedPoint.ipynb

In [ ]:
from typing import TypeVar, Set, FrozenSet, Dict, Tuple, Optional

S = TypeVar('S')
T = TypeVar('T')
K = TypeVar('K')
V = TypeVar('V')

In [ ]:
%mypy
def arb(M: Set[T]) -> T:
    for x in M:
        return x
    assert False, 'Error: arb called with empty set!'

The function `cart_prod(A, B)` computes the <em style="color:blue">Cartesian product</em> $A \times B$ of the sets $A$ and $B$ where
$$ A \times B := \{ (x, y) \mid x \in A \wedge y \in B \}. $$

In [ ]:
%mypy
def cart_prod(A: Set[S], B: Set[T]) -> Set[Tuple[S, T]]:
    return { (x, y) for x in A for y in B }

This function takes three arguments:
- `Pairs`  a set `Pairs` of pairs of states from some <span style="font-variant:small-caps;">Fsm</span> $F$.
   If $(p_1, p_2) \in \texttt{Pairs}$, then $p_1$ and $p_2$ are known to be *separable*,
- `States` is the set of states of the <span style="font-variant:small-caps;">Fsm</span> $F$,
- `Sigma`  is the alphabet of the underlying <span style="font-variant:small-caps;">Fsm</span> $F$.

The function `separate(Pairs)` computes the set of pairs of states $(q_1, q_2)$ that are separable because there is some
$c \in \Sigma$ such that 
$$\delta(q_1,c) = p_1, \quad \delta(q_2,c) = p_2. $$

In [ ]:
%mypy
def separate(Pairs: Set[Tuple[T, T]], 
             States: Set[T], 
             Sigma: Set[str], 
             delta: Dict[Tuple[T, str], T]
            ) -> Set[Tuple[T, T]]:
    return { (q1, q2) for q1 in States
                      for q2 in States
                      for c in Sigma 
                      if (delta[q1, c], delta[q2, c]) in Pairs
           }

Given a state $p$, find the equivalence class of $p$.  Here, `eqClasses` is a set of sets of states.

In [ ]:
%mypy
def find_equivalence_class(p: T, eqClasses: Set[FrozenSet[T]]) -> FrozenSet[T]:
    return arb({ C for C in eqClasses if p in C })

The function `minimize(A)` takes a deterministic 
<span style="font-variant:small-caps;">Fsm</span> `F` as its input.
Here `F` is a 5-tuple of the form
$$ F = (Q, \Sigma, \delta, q_0, A) $$
The algorithm performs the following steps:
1. All unreachable states are eliminated.
2. All accepting states are separated form all non-accepting states.
3. States are separated as long as possible.
   Two states $p_1$ and $p_2$ are separable if there is a character 
   $c \in \Sigma$ such that 
   $$\delta(p_1,c) = q_1, \quad \delta(p_2,c) = q_2, \quad \textrm{and} \quad
     \mbox{$q_1$ and $q_2$ are separable.}
   $$
4. States that are not separable are *equivalent* and are therefore identified and grouped
   in equivalence classes. 

In [ ]:
%mypy
FSM = Tuple[Set[T], Set[str], Dict[Tuple[T, str], T], T, Set[T]]
def minimize(F: FSM) -> FSM:
    States, Sigma, delta, q0, Accepting = F
    States       = fixpoint({q0}, lambda q: { delta[q, c] for c in Sigma })
    Separable    = cart_prod(States - Accepting, Accepting) | \
                   cart_prod(Accepting, States - Accepting)
    AllSeparable = fixpoint(Separable, lambda Pairs: separate(Pairs, States, Sigma, delta))
    Equivalent   = cart_prod(States, States) - AllSeparable
    EquivClasses = { frozenset({ p for p in States if (p, q) in Equivalent })
                     for q in States 
                   }
    newQ0        = arb({ M for M in EquivClasses if q0 in M })
    newAccept    = { M for M in EquivClasses if arb(M) in Accepting }   
    newDelta     = {}
    for q in States:
        for c in Sigma:
            p = delta.get((q, c))
            if p != None:
                classOfP = find_equivalence_class(p, EquivClasses)
                classOfQ = find_equivalence_class(q, EquivClasses)
                newDelta[(classOfQ, c)] = classOfP
    return EquivClasses, Sigma, newDelta, newQ0, newAccept